In [1]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))

In [2]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                ssCnt[can] = ssCnt.get(can, 0) + 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData

In [3]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [4]:
C1 = createC1(dataSet)

In [5]:
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [6]:
D = list(map(set, dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [7]:
L1, suppData0 = scanD(D, C1, 0.5)

In [9]:
L1
#suppData0

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [25]:
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1, L2 = list(Lk[i])[:k - 2], list(Lk[j])[:k - 2]
            L1.sort()
            L2.sort()
            print('L1:', L1)
            print('L2:', L2)
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
                print('retList:', retList)
    return retList

In [26]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [27]:
L, suppData = apriori(dataSet)
L

L1: []
L2: []
retList: [frozenset({2, 5})]
L1: []
L2: []
retList: [frozenset({2, 5}), frozenset({3, 5})]
L1: []
L2: []
retList: [frozenset({2, 5}), frozenset({3, 5}), frozenset({1, 5})]
L1: []
L2: []
retList: [frozenset({2, 5}), frozenset({3, 5}), frozenset({1, 5}), frozenset({2, 3})]
L1: []
L2: []
retList: [frozenset({2, 5}), frozenset({3, 5}), frozenset({1, 5}), frozenset({2, 3}), frozenset({1, 2})]
L1: []
L2: []
retList: [frozenset({2, 5}), frozenset({3, 5}), frozenset({1, 5}), frozenset({2, 3}), frozenset({1, 2}), frozenset({1, 3})]
L1: [2]
L2: [3]
L1: [2]
L2: [2]
retList: [frozenset({2, 3, 5})]
L1: [2]
L2: [1]
L1: [3]
L2: [2]
L1: [3]
L2: [1]
L1: [2]
L2: [1]


[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [28]:
aprioriGen(L[1], 3)

L1: [2]
L2: [3]
L1: [2]
L2: [2]
retList: [frozenset({2, 3, 5})]
L1: [2]
L2: [1]
L1: [3]
L2: [2]
L1: [3]
L2: [1]
L1: [2]
L2: [1]


[frozenset({2, 3, 5})]

In [56]:
L, suppData = apriori(dataSet, minSupport = 0.5)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [29]:
suppData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

In [41]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:    #freqSet是已找出的高频item之一
            H1 = [frozenset([item]) for item in freqSet]    #H是该高频item中的长度为一subset
            print('H1:', H1)
            if (i > 1):    #略复杂情况下，freqSet中item长度大于2
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:    #情况简单，len为1的item-->len为1的item
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
            print('prunedH:', prunedH)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    print('rulesFromConseq, H:', H)
    print('rulesFromConseq, freqSet:', freqSet)
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)    #create len == m+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)    #find the rules which satisfies the minConf
        print('rulesFromConseq, Hmp1:', Hmp1)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)    #try more possibilities

In [42]:
rules = generateRules(L, suppData, minConf = 0.5)

H1: [frozenset({2}), frozenset({3})]
frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
prunedH: [frozenset({2})]
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
prunedH: [frozenset({2}), frozenset({3})]
H1: [frozenset({3}), frozenset({5})]
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
prunedH: [frozenset({3})]
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
prunedH: [frozenset({3}), frozenset({5})]
H1: [frozenset({2}), frozenset({5})]
frozenset({5}) --> frozenset({2}) conf: 1.0
prunedH: [frozenset({2})]
frozenset({2}) --> frozenset({5}) conf: 1.0
prunedH: [frozenset({2}), frozenset({5})]
H1: [frozenset({1}), frozenset({3})]
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
prunedH: [frozenset({1})]
frozenset({1}) --> frozenset({3}) conf: 1.0
prunedH: [frozenset({1}), frozenset({3})]
H1: [frozenset({2}), frozenset({3}), frozenset({5})]
rulesFromConseq, H: [frozenset({2}), frozenset({3}), frozenset({5})]
rulesFromConseq, freqSet: froze

In [36]:
rules

[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

In [66]:
mushDatSet = [line.split() for line in open('mushroom.dat').readlines()]
mushDatSet

[['1',
  '3',
  '9',
  '13',
  '23',
  '25',
  '34',
  '36',
  '38',
  '40',
  '52',
  '54',
  '59',
  '63',
  '67',
  '76',
  '85',
  '86',
  '90',
  '93',
  '98',
  '107',
  '113'],
 ['2',
  '3',
  '9',
  '14',
  '23',
  '26',
  '34',
  '36',
  '39',
  '40',
  '52',
  '55',
  '59',
  '63',
  '67',
  '76',
  '85',
  '86',
  '90',
  '93',
  '99',
  '108',
  '114'],
 ['2',
  '4',
  '9',
  '15',
  '23',
  '27',
  '34',
  '36',
  '39',
  '41',
  '52',
  '55',
  '59',
  '63',
  '67',
  '76',
  '85',
  '86',
  '90',
  '93',
  '99',
  '108',
  '115'],
 ['1',
  '3',
  '10',
  '15',
  '23',
  '25',
  '34',
  '36',
  '38',
  '41',
  '52',
  '54',
  '59',
  '63',
  '67',
  '76',
  '85',
  '86',
  '90',
  '93',
  '98',
  '107',
  '113'],
 ['2',
  '3',
  '9',
  '16',
  '24',
  '28',
  '34',
  '37',
  '39',
  '40',
  '53',
  '54',
  '59',
  '63',
  '67',
  '76',
  '85',
  '86',
  '90',
  '94',
  '99',
  '109',
  '114'],
 ['2',
  '3',
  '10',
  '14',
  '23',
  '26',
  '34',
  '36',
  '39',
  '41',
 

In [69]:
L, suppData = apriori(mushDatSet, minSupport = 0.3)
for item in L[3]:
    if item.intersection('2'):
        print(item)

frozenset({'34', '59', '28', '2'})
frozenset({'34', '28', '2', '85'})
frozenset({'34', '28', '2', '90'})
frozenset({'34', '2', '28', '39'})
frozenset({'2', '63', '28', '39'})
frozenset({'34', '63', '28', '2'})
frozenset({'59', '63', '28', '2'})
frozenset({'63', '28', '2', '85'})
frozenset({'86', '63', '28', '2'})
frozenset({'59', '28', '2', '85'})
frozenset({'86', '2', '28', '39'})
frozenset({'86', '34', '28', '2'})
frozenset({'86', '59', '28', '2'})
frozenset({'86', '28', '2', '85'})
frozenset({'86', '28', '2', '90'})
frozenset({'59', '28', '2', '90'})
frozenset({'85', '28', '2', '90'})
frozenset({'2', '59', '28', '39'})
frozenset({'2', '28', '39', '85'})
frozenset({'2', '28', '39', '90'})
frozenset({'28', '34', '53', '2'})
frozenset({'28', '53', '2', '85'})
frozenset({'28', '53', '2', '90'})
frozenset({'28', '2', '53', '39'})
frozenset({'34', '53', '2', '85'})
frozenset({'34', '53', '2', '90'})
frozenset({'34', '2', '53', '39'})
frozenset({'86', '28', '53', '2'})
frozenset({'86', '2'